In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
data_path = '../input/cassava-leaf-disease-classification'
print(len(os.listdir('../input/cassava-leaf-disease-classification/train_images')))
print(len(os.listdir('../input/cassava-leaf-disease-classification/test_images')))

In [ ]:
import pandas as pd
train_df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train_df.head()

In [ ]:
import json
import seaborn as sns
with open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json') as file:
    print(json.dumps(json.loads(file.read()), indent=4))
sns.countplot(x = 'label', data = train_df)
train_df['label'].value_counts()

In [ ]:
import cv2
import matplotlib.pyplot as plt
from PIL import Image

# def get_img(path):
#     im_bgr = cv2.imread(path)
#     im_rgb = im_bgr[:, :, ::-1]
#     return im_rgb

# a = get_img('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
Image.open('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
# print(a.shape)
# plt.imshow(a)

In [ ]:
from sklearn.model_selection import train_test_split
data_list = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train_list, valid_list = train_test_split(data_list, test_size=0.2, random_state=42)
print(data_list)
print(train_list.shape)
print(valid_list.shape)


In [ ]:

from torch.utils.data import Dataset,DataLoader
from PIL import Image
class CassavaDataset(Dataset):
    
    def __init__(self,  df, transforms=None, target_transform=None):
        super().__init__()
        self.data_root = "../input/cassava-leaf-disease-classification/train_images"
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.target_transform = target_transform
 
    def __getitem__(self, index: int):
        img_path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
#         img  = get_img(img_path)
        img = Image.open(img_path).convert('RGB')
        
        target = self.df.iloc[index]['label']

        if self.transforms is not None:
            img = self.transforms(img)
            
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target
    
    def __len__(self):
        return len(self.df)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms


def Get_Dataloader(train_list, valid_list):
    
    # data augmentation is more abundant for training data than validation data
    train_transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(224), # we will use resnet34, which requires the size of input to be 224*224*3, you can also change the network structure to accomodate the original image size
            transforms.ColorJitter(0.2, 0.2, 0.2),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.5,0.5,0.5], std = [0.5,0.5,0.5]),
        ])
    
    valid_transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.5,0.5,0.5], std = [0.5,0.5,0.5]),
        ])
 
    # training and validation datasets
    train_data = CassavaDataset(train_list, transforms=train_transform, target_transform=None)
    valid_data = CassavaDataset(valid_list, transforms=valid_transform, target_transform=None)
    
    # training and validation dataloaders
    train_loader = torch.utils.data.DataLoader(
        train_data,
        batch_size=64,
        num_workers=2,
        shuffle=True, 
        pin_memory=True,
    )

    valid_loader = torch.utils.data.DataLoader(
        valid_data, 
        batch_size=64,
        num_workers=2,
        shuffle=False,
        pin_memory=True,
    )
    
    return train_loader, valid_loader

train_loader, valid_loader = Get_Dataloader(train_list, valid_list)

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
def save_model(model, save_file):
    state = {
        'model': model.state_dict(),
    }
    torch.save(state, save_file)
    del state
    
def adjust_learning_rate(optimizer, epoch, lr, milestones):
    '''decrease learning rate lr to 0.1*lr when train for milestone[0]th epoch, and decrease it to 0.01*lr when comes to milestone[1]th epoch
    '''
    lr_1 = lr*(0.1**(epoch>=milestones[0]))*(0.1**(epoch>=milestones[1]))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr_1

    return lr_1


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score
def train(model, criterion, optimizer, train_loader, device, epoch):
    losses = AverageMeter()
    accs = AverageMeter()
    
    model.train()
    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        bsz = labels.shape[0]
        
        output = model(images)
        loss = criterion(output, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(output.data, 1)
        acc = accuracy_score(labels.cpu(), predicted.cpu())
        
        accs.update(acc)
        losses.update(loss.item())
        
    return accs.avg, losses.avg

In [ ]:
def valid(model, criterion, valid_loader, device, epoch):
    losses = AverageMeter()
    accs = AverageMeter()
    
    model.eval()
    with torch.no_grad():
        for idx, (images, labels) in enumerate(valid_loader):
            images = images.to(device)
            labels = labels.to(device)
            bsz = labels.shape[0]
        
            output = model(images)
            loss = criterion(output, labels)
        
            _, predicted = torch.max(output.data, 1)
            acc = accuracy_score(labels.cpu(), predicted.cpu())
        
            accs.update(acc)
            losses.update(loss.item())
      
    return accs.avg, losses.avg

In [ ]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

num_classes = 5
model = models.resnet50() # we use resnet34 as the backbone
#model = SwinTransformer(
#    num_classes = 5
#)
model.fc = nn.Linear(model.fc.in_features, num_classes) # we should change the last fc layer of the predefined resnet34 network to accomodate our classification problem
lr = 0.1
optimizer = optim.SGD(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
n_epochs = 80 # just a demo
milestones = [30, 40]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.to(device)
train_loader, valid_loader = Get_Dataloader(train_list, valid_list)
os.makedirs('cassava/model')

train_losses = []
train_acces = []
eval_losses = []
eval_acces = []


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.2, 
patience=3, verbose=True, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0.0001, eps=1e-08)


early_stopping = EarlyStopping(patience=8, verbose=True)

best_acc = 0
for epoch in range(1, n_epochs+1):
    #lr_now = adjust_learning_rate(optimizer, epoch, lr, milestones) # adjust learning rate
    acc_train, loss_train = train(model, criterion, optimizer, train_loader, device, epoch)
    #print("Epoch[{}], Train acc: {:.3f}, Train loss: {:.3f}, lr {:.3f}.".format(epoch, acc_train, loss_train, lr_now))
    print("Epoch[{}], Train acc: {:.3f}, Train loss: {:.3f}.".format(epoch, acc_train, loss_train))
    acc_val, loss_val = valid(model, criterion, valid_loader, device, epoch)
    scheduler.step(epoch)
    print("Epoch[{}], Valid acc: {:.3f}, Valid loss: {:.3f}.".format(epoch, acc_val, loss_val))
    train_losses.append(loss_train)
    train_acces.append(acc_train)
    eval_losses.append(loss_val)
    eval_acces.append(acc_val)
    if acc_val > best_acc:
        save_model(model, 'cassava/model/best.pth') # save model
        best_acc = acc_val
    early_stopping(loss_val, model)
        
    if early_stopping.early_stop:
        print("Early stopping")
        break

In [ ]:
plt.plot(np.arange(len(train_losses)), train_losses,label="train loss")
plt.xlabel('epoches')
plt.title('train loss')
plt.show()

In [ ]:

plt.plot(np.arange(len(train_acces)), train_acces, label="train acc")

plt.legend()
plt.xlabel('epoches')
plt.title('train acc')
plt.show()

In [ ]:

plt.plot(np.arange(len(eval_losses)), eval_losses, label="valid loss")

plt.legend()
plt.xlabel('epoches')
plt.title('valid loss')
plt.show()

In [ ]:

plt.plot(np.arange(len(eval_acces)), eval_acces, label="valid acc")
plt.legend()
plt.xlabel('epoches')
plt.title('valid acc')
plt.show()

In [ ]:
from glob import glob
import pandas as pd
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import torchvision.transforms as transforms

# def a new dataset function
class CassavaDataset_test(Dataset):
    
    def __init__(self,  df, transforms=None, target_transform=None):
        super().__init__()
        self.data_root = "/kaggle/input/cassava-leaf-disease-classification/test_images"
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.target_transform = target_transform
 
    def __getitem__(self, index: int):
        img_path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
#         img  = get_img(img_path)
        img = Image.open(img_path).convert('RGB')
        
        img_id = self.df.iloc[index]['image_id']      # add
        target = self.df.iloc[index]['label']

        if self.transforms is not None:
            img = self.transforms(img)
            
        if self.target_transform is not None:
            target = self.target_transform(target)

        return img_id, img, target                   # add
    
    def __len__(self):
        return len(self.df)

    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load model
model_file = 'cassava/model/best.pth'
model = models.resnet50()
model.fc = nn.Linear(model.fc.in_features, 5)
#model = SwinTransformer(
#    num_classes = 5
#)
checkpoint = torch.load(model_file)
state = checkpoint['model']
model.load_state_dict(state)
model.to(device)


# test image loader
result = {}
test_images = glob('/kaggle/input/cassava-leaf-disease-classification/test_images/*.jpg')
for img in test_images:
    img_name = img.split('/')[-1]
    if img_name not in result:
        result[img_name] = 99
test_list = pd.DataFrame.from_dict(result, orient='index', columns=['label']).reset_index().rename(columns={'index': 'image_id'})

valid_transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor(), transforms.Normalize(mean = [0.5,0.5,0.5], std = [0.5,0.5,0.5]),])
test_data = CassavaDataset_test(test_list, transforms=valid_transform, target_transform=None)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=16,num_workers=2,shuffle=False,pin_memory=True)

#  use model to test image and produce a 'submission.csv' file
model.eval()
with torch.no_grad():
    for idx, (image_id, images, _) in enumerate(test_loader):
        images = images.to(device)
        bsz = images.shape[0]
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        for i in range(bsz):
            result[image_id[i]] = predicted[i].item()  # record predicted label
        
result = pd.DataFrame.from_dict(result, orient='index', columns=['label']).reset_index().rename(columns={'index': 'image_id'})
result.to_csv('submission.csv', index=False)